En este archivo vamos a preprocesar el texto de manera que saquemos las etiquetas de cada tipo de palabra (POS tagging). Después pasaremos texto + etiquetas a un cierto formato, que será el que le pasemos al modelo entrenado para que resuelva las correferencias en el texto. Cabe destacar que el texto que vamos a usar está en español.

### Al terminar, sacar hacer el requisitos.txt

In [1]:
%pip install -r requirements.txt

  Using cached ufal_udpipe-1.4.0.1-cp311-cp311-win_amd64.whl.metadata (11 kB)
Using cached ufal_udpipe-1.4.0.1-cp311-cp311-win_amd64.whl (872 kB)
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
%pip freeze > requirements.txt

Note: you may need to restart the kernel to use updated packages.


# 1 Sacar etiquetas
Como hemos mencionado antes, primero vamos a sacar las etiquetas de cada palabra en un texto, comprobaremos el funcionamiento de varios modelos y discutiremos con cuál nos quedamos.

## 1.1 Textos a etiquetar
El primer paso es definir los textos de los que vamos a sacar las etiquetas. Hemos recogido 12 frases cortas para ver el funcionamiento de los modelos, creemos que es suficiente para elegir un modelo.

In [22]:
texts = [
    'El gato come pescado.',
    'El equipo de fútbol jugó su partido. Este ganó con facilidad.',
    'El coche rojo se estropeó, así que lo llevé al taller.',
    'La presidenta y el director se reunieron; ella habló primero.',
    'Entregué el informe a la jefa después de que ella lo leyera.',
    'Los equipos trabajaron duro, y al final ellos ganaron el premio.',
    'A pesar de sus problemas, el artista terminó su obra.',
    'El hermano de María dijo que él vendría.',
    'En su oficina, el abogado revisó los documentos.',
    'El libro que leí es fascinante; este autor siempre sorprende.',
    'El gato persiguió al ratón, pero este logró escapar.',
    'Hablé con Pedro sobre su proyecto y luego él me envió los archivos.',
    'Ayer hablé con Juan, le dije: "dímelo, por favor", y no me lo quiso decir',
    'Dímelo',
    'Dí me lo',
    'Di me lo'
]

## 1.2 Modelos
Antes de pasar a la acción vamos a definir los imports pertinentes y descargar algún modelo si fuera necesario

In [2]:
import spacy
import stanza

c:\Users\Jorge\Documents\Proyectos de programación\PLN\Desambiguador-correferencias\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
spacy.cli.download('es_core_news_sm')
print('-'*80)
spacy.cli.download('es_dep_news_trf')
print('-'*80)
stanza.download('es')

✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
--------------------------------------------------------------------------------
✔ Download and installation successful
You can now load the package via spacy.load('es_dep_news_trf')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
--------------------------------------------------------------------------------


2025-12-23 17:03:11 INFO: Downloaded file to C:\Users\Jorge\stanza_resources\resources.json
2025-12-23 17:03:11 INFO: Downloading default packages for language: es (Spanish) ...
2025-12-23 17:03:15 INFO: File exists: C:\Users\Jorge\stanza_resources\es\default.zip
2025-12-23 17:03:19 INFO: Finished downloading models and saved to C:\Users\Jorge\stanza_resources


## 1.3 Pruebas
Vamos a empezar a probar modelos sobre todas las frases que hemos definido antes.

### 1.3.1 Spacy - es_core_news_sm

In [10]:
nlp = spacy.load("es_core_news_sm")  # Para español
docs = []
for text in texts: docs.append(nlp(text))
for doc in docs:
    for token in doc:
        print(token.text, token.pos_)
    print()

El DET
gato NOUN
come VERB
pescado ADJ
. PUNCT

El DET
equipo NOUN
de ADP
fútbol NOUN
jugó VERB
su DET
partido NOUN
. PUNCT
Este PRON
ganó VERB
con ADP
facilidad NOUN
. PUNCT

El DET
coche NOUN
rojo ADJ
se PRON
estropeó VERB
, PUNCT
así ADV
que SCONJ
lo PRON
llevé VERB
al ADP
taller NOUN
. PUNCT

La DET
presidenta NOUN
y CCONJ
el DET
director NOUN
se PRON
reunieron VERB
; PUNCT
ella PRON
habló VERB
primero ADV
. PUNCT

Entregué PROPN
el DET
informe NOUN
a ADP
la DET
jefa NOUN
después ADV
de ADP
que SCONJ
ella PRON
lo PRON
leyera VERB
. PUNCT

Los DET
equipos NOUN
trabajaron VERB
duro ADJ
, PUNCT
y CCONJ
al ADP
final NOUN
ellos PRON
ganaron VERB
el DET
premio NOUN
. PUNCT

A ADP
pesar NOUN
de ADP
sus DET
problemas NOUN
, PUNCT
el DET
artista NOUN
terminó VERB
su DET
obra NOUN
. PUNCT

El DET
hermano NOUN
de ADP
María PROPN
dijo VERB
que SCONJ
él PRON
vendría VERB
. PUNCT

En ADP
su DET
oficina NOUN
, PUNCT
el DET
abogado NOUN
revisó VERB
los DET
documentos NOUN
. PUNCT

El DET
libro NOU

En general lo hace muy bien, aunque confunde VERB que comienzan una oración con PROPN (sustantivos propios) y, en la primera oración, confunde el sustantivo pescado por el adjetivo, por lo que podemos intuir que cometerá más veces ese error. Por lo que a nuestra futura tarea respecta, el primer error puede ser garrafal, pues podría pensar el modelo que un pronombre se refiere a un verbo de esos mal etiquetados. Además, el verbo decir más los enclíticos me y lo lo considera un sustantivo, fallo enorme ya que son dos pronombres tras el verbo.
Veamos el resto de modelos.

### 1.3.2 Spacy - es_dep_news_trf

In [11]:
nlp = spacy.load("es_dep_news_trf")
docs = []
for text in texts: docs.append(nlp(text))
for doc in docs:
    for token in doc:
        print(token.text, token.pos_)
    print()

El DET
gato NOUN
come VERB
pescado NOUN
. PUNCT

El DET
equipo NOUN
de ADP
fútbol NOUN
jugó VERB
su DET
partido NOUN
. PUNCT
Este PRON
ganó VERB
con ADP
facilidad NOUN
. PUNCT

El DET
coche NOUN
rojo ADJ
se PRON
estropeó VERB
, PUNCT
así ADV
que SCONJ
lo PRON
llevé VERB
al ADP
taller NOUN
. PUNCT

La DET
presidenta NOUN
y CCONJ
el DET
director NOUN
se PRON
reunieron VERB
; PUNCT
ella PRON
habló VERB
primero ADV
. PUNCT

Entregué VERB
el DET
informe NOUN
a ADP
la DET
jefa NOUN
después ADV
de ADP
que SCONJ
ella PRON
lo PRON
leyera VERB
. PUNCT

Los DET
equipos NOUN
trabajaron VERB
duro ADV
, PUNCT
y CCONJ
al ADP
final NOUN
ellos PRON
ganaron VERB
el DET
premio NOUN
. PUNCT

A ADP
pesar NOUN
de ADP
sus DET
problemas NOUN
, PUNCT
el DET
artista NOUN
terminó VERB
su DET
obra NOUN
. PUNCT

El DET
hermano NOUN
de ADP
María PROPN
dijo VERB
que SCONJ
él PRON
vendría VERB
. PUNCT

En ADP
su DET
oficina NOUN
, PUNCT
el DET
abogado NOUN
revisó VERB
los DET
documentos NOUN
. PUNCT

El DET
libro NOU

En este caso vemos que corrige todos los errores previos, salvo el caso de los enclíticos, que los considera verbos (bien, pero solo la raíz es el verbo). Esto muestra que este modelo es mejor, pero no perfecto, pues confunde, por ejemplo "Dí", de separar los enclíticos en "Dímelo", por AUX, pero sin la tilde dice correctamente que es un verbo, esto lo tendremos en cuenta más adelante.

### 1.3.3 Spacy - es_dep_news_trf (texto en minúsculas)

In [23]:
nlp = spacy.load("es_dep_news_trf")
docs = []
for text in texts: docs.append(nlp(text.lower()))
for doc in docs:
    for token in doc:
        print(token.text, token.pos_)
    print()

el DET
gato NOUN
come VERB
pescado NOUN
. PUNCT

el DET
equipo NOUN
de ADP
fútbol NOUN
jugó VERB
su DET
partido NOUN
. PUNCT
este PRON
ganó VERB
con ADP
facilidad NOUN
. PUNCT

el DET
coche NOUN
rojo ADJ
se PRON
estropeó VERB
, PUNCT
así ADV
que SCONJ
lo PRON
llevé VERB
al ADP
taller NOUN
. PUNCT

la DET
presidenta NOUN
y CCONJ
el DET
director NOUN
se PRON
reunieron VERB
; PUNCT
ella PRON
habló VERB
primero ADV
. PUNCT

entregué VERB
el DET
informe NOUN
a ADP
la DET
jefa NOUN
después ADV
de ADP
que SCONJ
ella PRON
lo PRON
leyera VERB
. PUNCT

los DET
equipos NOUN
trabajaron VERB
duro ADV
, PUNCT
y CCONJ
al ADP
final NOUN
ellos PRON
ganaron VERB
el DET
premio NOUN
. PUNCT

a ADP
pesar NOUN
de ADP
sus DET
problemas NOUN
, PUNCT
el DET
artista NOUN
terminó VERB
su DET
obra NOUN
. PUNCT

el DET
hermano NOUN
de ADP
maría PROPN
dijo VERB
que SCONJ
él PRON
vendría VERB
. PUNCT

en ADP
su DET
oficina NOUN
, PUNCT
el DET
abogado NOUN
revisó VERB
los DET
documentos NOUN
. PUNCT

el DET
libro NOU

Al hacer varias pruebas nos damos cuenta que estos modelos son <em>case sensitive</em>, es decir, las palabras en mayúsculas y minúsculas importan. En este caso vemos que se solventa el problema del AUX en "dí", pero aún así palabras como "dímelo" no las hace bien. Probemos con otro modelo, en este caso Stanza.

### 1.3.4 Stanza

In [13]:
nlp = stanza.Pipeline('es', verbose=False)
docs = []
for text in texts: docs.append(nlp(text))
for doc in docs:
    for sentence in doc.sentences:
        for word in sentence.words:
            print(word.text, word.pos)
    print()

El DET
gato NOUN
come VERB
pescado NOUN
. PUNCT

El DET
equipo NOUN
de ADP
fútbol NOUN
jugó VERB
su DET
partido NOUN
. PUNCT
Este PRON
ganó VERB
con ADP
facilidad NOUN
. PUNCT

El DET
coche NOUN
rojo ADJ
se PRON
estropeó VERB
, PUNCT
así ADV
que SCONJ
lo PRON
llevé VERB
a ADP
el DET
taller NOUN
. PUNCT

La DET
presidenta NOUN
y CCONJ
el DET
director NOUN
se PRON
reunieron VERB
; PUNCT
ella PRON
habló VERB
primero ADV
. PUNCT

Entregué VERB
el DET
informe NOUN
a ADP
la DET
jefa NOUN
después ADV
de ADP
que SCONJ
ella PRON
lo PRON
leyera VERB
. PUNCT

Los DET
equipos NOUN
trabajaron VERB
duro ADJ
, PUNCT
y CCONJ
a ADP
el DET
final NOUN
ellos PRON
ganaron VERB
el DET
premio NOUN
. PUNCT

A ADP
pesar NOUN
de ADP
sus DET
problemas NOUN
, PUNCT
el DET
artista NOUN
terminó VERB
su DET
obra NOUN
. PUNCT

El DET
hermano NOUN
de ADP
María PROPN
dijo VERB
que SCONJ
él PRON
vendría VERB
. PUNCT

En ADP
su DET
oficina NOUN
, PUNCT
el DET
abogado NOUN
revisó VERB
los DET
documentos NOUN
. PUNCT

El D

En este caso el modelo funciona muy similar a Spacy con es_dep_news_trf. Tampoco identifica correctamente los pronombres en "dímelo", y en este caso lo marca como interjección (INTJ). Cuando solo ponemos el verbo con enclíticos (Dímelo) lo considera verbo, y cuando lo separamos con espacios las partes analiza bien los pronombres pero el verbo vuelve a confundirlo con interjección, aunque si el verbo está sin tilde lo sigue marcanto como INTJ. Probemos con el texto en minúsculas. 

### 1.3.5 Stanza (minúsculas)

In [14]:
nlp = stanza.Pipeline('es', verbose=False)
docs = []
for text in texts: docs.append(nlp(text.lower()))
for doc in docs:
    for sentence in doc.sentences:
        for word in sentence.words:
            print(word.text, word.pos)
    print()

el DET
gato NOUN
come VERB
pescado NOUN
. PUNCT

el DET
equipo NOUN
de ADP
fútbol NOUN
jugó VERB
su DET
partido NOUN
. PUNCT
este PRON
ganó VERB
con ADP
facilidad NOUN
. PUNCT

el DET
coche NOUN
rojo ADJ
se PRON
estropeó VERB
, PUNCT
así ADV
que SCONJ
lo PRON
llevé VERB
a ADP
el DET
taller NOUN
. PUNCT

la DET
presidenta NOUN
y CCONJ
el DET
director NOUN
se PRON
reunieron VERB
; PUNCT
ella PRON
habló VERB
primero ADV
. PUNCT

entregué VERB
el DET
informe NOUN
a ADP
la DET
jefa NOUN
después ADV
de ADP
que SCONJ
ella PRON
lo PRON
leyera VERB
. PUNCT

los DET
equipos NOUN
trabajaron VERB
duro ADJ
, PUNCT
y CCONJ
a ADP
el DET
final NOUN
ellos PRON
ganaron VERB
el DET
premio NOUN
. PUNCT

a ADP
pesar NOUN
de ADP
sus DET
problemas NOUN
, PUNCT
el DET
artista NOUN
terminó VERB
su DET
obra NOUN
. PUNCT

el DET
hermano NOUN
de ADP
maría NOUN
dijo VERB
que SCONJ
él PRON
vendría VERB
. PUNCT

en ADP
su DET
oficina NOUN
, PUNCT
el DET
abogado NOUN
revisó VERB
los DET
documentos NOUN
. PUNCT

el DE

Aquí cambia ligeramente: "di" lo marca bien como verbo, mientras que "dímelo" lo marca como sustantivo, y lo demás se mantiene igual. Aunque sea parecido a Spacy, este parece menos consistente con verbos con enclíticos. Ahora probamos con el último modelo, Flair.

### 1.3.6 Flair

In [19]:
from flair.data import Sentence
from flair.models import SequenceTagger

# Cargar modelo POS (multilingual)
tagger = SequenceTagger.load("pos-multi")

for text in texts:
    sentence = Sentence(text.lower())
    tagger.predict(sentence)
    for token in sentence:
        print(token.text, token.get_labels()[0].value)
    print()

2025-12-23 17:24:08,419 SequenceTagger predicts: Dictionary with 17 tags: NOUN, PUNCT, ADP, VERB, ADJ, DET, PROPN, ADV, PRON, AUX, CCONJ, NUM, SCONJ, PART, X, SYM, INTJ
el DET
gato NOUN
come VERB
pescado NOUN
. PUNCT

el DET
equipo NOUN
de ADP
fútbol NOUN
jugó VERB
su DET
partido NOUN
. PUNCT
este PRON
ganó VERB
con ADP
facilidad NOUN
. PUNCT

el DET
coche NOUN
rojo ADJ
se PRON
estropeó VERB
, PUNCT
así ADV
que CCONJ
lo PRON
llevé VERB
al DET
taller NOUN
. PUNCT

la DET
presidenta NOUN
y CCONJ
el DET
director NOUN
se PRON
reunieron VERB
; PUNCT
ella PRON
habló VERB
primero ADJ
. PUNCT

entregué VERB
el DET
informe NOUN
a ADP
la DET
jefa NOUN
después ADV
de ADP
que SCONJ
ella PRON
lo PRON
leyera VERB
. PUNCT

los DET
equipos NOUN
trabajaron VERB
duro ADJ
, PUNCT
y CCONJ
al ADP
final NOUN
ellos PRON
ganaron VERB
el DET
premio NOUN
. PUNCT

a ADP
pesar NOUN
de ADP
sus DET
problemas NOUN
, PUNCT
el DET
artista NOUN
terminó VERB
su DET
obra NOUN
. PUNCT

el DET
hermano NOUN
de ADP
maría NOU

En este modelo vemos más inconsistencias todavía. "di" lo marca como ADP, y "lo" al separar "dímelo" lo marca como verbo. No mejora en ningún aspecto a los anteriores. 

## 1.4 Preprocesamiento custom
Tras varias pruebas pensamos que el Spacy con dep news (texto en minúsculas) es el más consistente, ya que solo falla en los enclíticos. Para mejorar con los enclíticos podemos hacer nuestro propio preprocesamiento al texto para que así lo detecte como debe. Para ello vamos a coger la lista de pronombres que pueden ir junto a los verbos, vamos a ponerlos en orden, e.g. nunca se tiene "lasme", siempre "melas", como en "dámelas". Pero esto solo no es suficiente, puesto que hay verbos que tienen "me", "te" u otros, e.g. come, comete. Pero comete es diferente de cómete, de comer, por lo que necesitamos una lista con verbos a los que no les tenemos que aplicar este preprocesamiento. Por otro lado no tenemos que tener todas las formas verbales de dichos verbos, puesto que solo buscamos aquellos cuyo sufijo pueda hacerse pasar por un pronombre enclítico, i.e. cometerían no nos interesa pues sabemos con certeza que no va a tener lugar a duda.

In [ ]:
# Lista de clíticos españoles
CLITICOS = ["me","te","se","lo","la","los","las","le","les","nos","os"]

# Orden típico permitido en español
# (simplificado pero suficientemente bueno para casi todos los casos)
ORDEN_CLITICOS = [
    ["me","lo"], ["me","la"], ["me","los"], ["me","las"],
    ["te","lo"], ["te","la"], ["te","los"], ["te","las"],
    ["se","lo"], ["se","la"], ["se","los"], ["se","las"],
    ["nos","lo"], ["nos","la"], ["nos","los"], ["nos","las"],
    ["os","lo"], ["os","la"], ["os","los"], ["os","las"],
]

# Generar patrones de clíticos pegados (ej: "melo", "telo", "selo")
PATRONES = sorted(
    [ "".join(p) for p in ORDEN_CLITICOS ] + CLITICOS,
    key=len, reverse=True
)

VERBOS = [
    "comete",
    "somete",
    "arremete",
    "promete",
    "remete",
    "entromete",
    "admite",
    "emite",
    "omite",
    "remite",
    "permite",
    "transmite",
    "dimite",
    "limite",
    "intermite",
    "retransmite",
    "siente",
    "consiente",
    "presiente",
    "resiente",
    "asiente",
    "disiente",
    "resume",
    "asume",
    "presume",
    "consume",
    "subsume",
    "come",
    "vale",
    'comeríamos'
] # Aún faltan muchísimos, igual mejor guardarlo como txt

In [ ]:
def separar_cliticos(token, lemma):
    """
    Separa clíticos de una forma verbal como 'dímelo', 'háblalo', 'cómetelo', etc.
    """
    t = token.lower()

    # Buscar si termina en un clítico válido (simple o doble)
    encontrados = []
    resto = t

    for patron in PATRONES:
        if resto.endswith(patron):
            encontrados.append(patron)          # ej: "melo"
            resto = resto[: -len(patron)]
            break

    # Si no encontró nada → no hay clíticos pegados
    if not encontrados:
        return [token]  

    # Convertir el verbo a su forma verbal (usar lemma como base)
    raiz = lemma

    # Normalizar la raíz para modo imperativo (caso típico)
    # Esto no es perfecto, pero funciona para mayoría:
    if raiz.endswith("r"):      # comer → come
        raiz = raiz[:-1]
    elif raiz.endswith("irse"): # "irse" → "ir" pero con reflexivos es complejo
        raiz = raiz[:-3]

    resultado = [raiz]

    # Ahora separar los clíticos (simple o doble)
    cl = encontrados[0]

    # Si es doble clítico ("melo", "selo", etc.)
    for c in CLITICOS:
        if cl.startswith(c):
            resto2 = cl[len(c):]
            if c in CLITICOS:
                resultado.append(c)
            if resto2 in CLITICOS:
                resultado.append(resto2)
            break
    
    return resultado